In [1]:
import pandas as pd
from scipy.spatial.distance import cosine

In [2]:
# movie_data = pd.read_csv("movie_updated.csv")
movie_data = pd.read_csv("imdb-top250.csv",delimiter="#")
movie_data.head(5)

,Id,Title,Year,Runtime,Languege,Country,Genre,Director,Writer,Actors,Production,IMDB_Rating
0,111161,The Shawshank Redemption,1994,142,English,USA,Crime|Drama,Frank Darabont,Stephen King|Frank Darabont,Tim Robbins|Morgan Freeman|Bob Gunton|William ...,Castle Rock Entertainment,9.3
1,68646,The Godfather,1972,175,English|Italian|Latin,USA,Crime|Drama,Francis Ford Coppola,Mario Puzo|Francis Ford Coppola|Mario Puzo,Marlon Brando|Al Pacino|James Caan|Richard S. ...,Paramount Pictures|Alfran Productions,9.2
2,71562,The Godfather: Part II,1974,202|220::(The Godfather Trilogy 1901-1980 VHS ...,English|Italian|Spanish|Latin|Sicilian,USA,Crime|Drama,Francis Ford Coppola,Francis Ford Coppola|Mario Puzo|Mario Puzo,Al Pacino|Robert Duvall|Diane Keaton|Robert De...,Paramount Pictures|The Coppola Company,9.0
3,468569,The Dark Knight,2008,152,English|Mandarin,USA|UK,Action|Crime|Drama|Thriller,Christopher Nolan,Jonathan Nolan|Christopher Nolan|Christopher N...,Christian Bale|Heath Ledger|Aaron Eckhart|Mich...,Warner Bros.|Legendary Entertainment|Syncopy|D...,9.0
4,50083,12 Angry Men,1957,96,English,USA,Crime|Drama,Sidney Lumet,Reginald Rose|Reginald Rose,Martin Balsam|John Fiedler|Lee J. Cobb|E.G. Ma...,Orion-Nova Productions,8.9


In [3]:
def get_all_types(genres):
    g_list = genres.tolist()
    genre = set()
    for x in g_list:
        y = x.split('|')
        genre.update(y)
    return list(genre)
    
def generate_dataset(movie_data):
    ### To get all genres
    genres = get_all_types(movie_data['Genre'])

    for g in genres:
        movie_data[g] = 0
    
    for i, row in movie_data.iterrows():
        g = row['Genre']
        g_list = g.split('|')
        for x in g_list:
            movie_data.set_value(i,x,1)

In [4]:
generate_dataset(movie_data)
# movie_data.columns
# movie_data.info()
### To remove duplicates
movie_data = movie_data.drop_duplicates(subset='Title', keep="last")
### To convert title to lower case
movie_data['Title'] = movie_data['Title'].str.lower()
# movie_data.head(5)

In [5]:
def get_movies_by_common_actors(vec1, data):
    cols = data.columns
    df_temp = pd.DataFrame(columns=cols)

    f1=vec1.Actors.split('|')
    for i, row in data.iterrows():
        f2 = row.Actors.split('|')
        flag = not set(f1).isdisjoint(f2)
        if flag:
            df_temp = df_temp.append(row)  
    return df_temp
        
def get_movies_by_common_directors(vec1, data):
    cols = data.columns
    df_temp = pd.DataFrame(columns=cols)

    f1=vec1.Director.split('|')
    for i, row in data.iterrows():
        f2 = row.Director.split('|')
        flag = not set(f1).isdisjoint(f2)
        if flag:
            df_temp = df_temp.append(row)
    return df_temp

def get_movies_by_common_writer(vec1, data):
    cols = data.columns
    df_temp = pd.DataFrame(columns=cols)

    f1=vec1.Writer.split('|')
    for i, row in data.iterrows():
        f2 = row.Writer.split('|')
        flag = not set(f1).isdisjoint(f2)
        if flag:
            df_temp = df_temp.append(row)
    return df_temp

def get_movies_by_common_production(vec1, data):
    cols = data.columns
    df_temp = pd.DataFrame(columns=cols)

    f1=vec1.Production.split('|')
    for i, row in data.iterrows():
        f2 = row.Production.split('|')
        flag = not set(f1).isdisjoint(f2)
        if flag:
            df_temp = df_temp.append(row)
    return df_temp

In [6]:
def similarity(row, vec2):
    s = 1 - cosine(row, vec2)
    return s

def recommand_movies(movie_data, movie):
    movie = movie.lower()
    vec1=movie_data[movie_data.Title==movie].iloc[0]
    df1 = get_movies_by_common_actors(vec1,movie_data)
    df2 = get_movies_by_common_directors(vec1,movie_data)
    df3 = get_movies_by_common_writer(vec1,movie_data)
    df4 = get_movies_by_common_writer(vec1,movie_data)

    cols = movie_data.columns
    movie_matched = pd.DataFrame(columns=cols)
    
    movie_matched = movie_matched.append(df1)
    movie_matched = movie_matched.append(df2)
    movie_matched = movie_matched.append(df3)
    movie_matched = movie_matched.append(df4)

    ### To remove duplicates
    movie_matched = movie_matched.drop_duplicates(subset='Title', keep="last")

    ### To remove duplicates
    movie_matched = movie_matched[movie_matched['Title'] != movie]

    ### features used to calculate similarity
    features = movie_matched.iloc[:,11:]
    vec2 = vec1.iloc[11:]

    sim = features.apply(similarity, args=(vec2,), axis=1)

    data = pd.DataFrame({"Title":movie_matched.Title, "similarity":sim})
    ### to sort according to similarity
    data = data.sort_values(['similarity','Title'], ascending=False)
    
    return data    

In [7]:
### movie --> queried movie
# movie="Pulp Fiction"
# movie="Shichinin No Samurai"
# movie="The Godfather"
# movie="Interstellar"
movie="3 idiots"
# movie="Dangal"

data=recommand_movies(movie_data,movie)
data.head(5)

,Title,similarity
249,pk,0.992848
234,munna bhai m.b.b.s.,0.992848
79,taare zameen par,0.986374
175,rang de basanti,0.986183
70,dangal,0.973787
